# ✅ A. Méthodes traditionnelles (orthographe + règles simples)

In [2]:
!pip install pyenchant

   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.9 MB ? eta -:--:--
    --------------------------------------- 0.3/11.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.9 MB 1.1 MB/s eta 0:00:11
   -- ------------------------------------- 0.8/11.9 MB 1.0 MB/s eta 0:00:11
   --- ------------------------------------ 1.0/11.9 MB 1.2 MB/s eta 0:00:09
   ----- ---------------------------------- 1.6/11.9 MB 1.4 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.9 MB 1.4 MB/s eta 0:00:07
   ------- -------------------------------- 2.4/11.9 MB 1.5 MB/s eta 0:00:07
   -------- ------------------------------- 2.6/11.9 MB 1.6 MB/s eta 0:00:06
   ---------- ----------------------------- 3.1/11.9 MB 1.7 MB/s eta 0:00:06
   ------------ --------------------------- 3.7/11.9 MB 1.7 MB/s eta 0:00:05
   -------------- ------------------------- 4.2/11.9 MB 1.8 MB/s eta 0:00:05
   --------------- -

In [ ]:
import enchant 
import re
import pandas as pd
def check_spelling_and_rules(text):
    d = enchant.Dict("en_US")
    words = re.findall(r"\b\w+\b", text)
    anomalies = [word for word in words if not d.check(word)]
    return anomalies
data = pd.read_csv("sephora_skincare_reviews.csv")

data['spelling_anomalies'] = data['feedback'].apply(check_spelling_and_rules)
print(data[['feedback','spelling_anomalies']].head(10))


                                            feedback  \
0  Noticed peeling when I used it with other acti...   
1  TThe pump drspensses too much prduct each time...   
2  Did ont suit my oily-ocmbination sin; mdaee it...   
3  It clogged my pors adn causeed breakouts near ...   
4  Packoging is convenient atd hygienic, pump wor...   
5  It clgoged my pores and cused breakouts near m...   
6  Packaginng is convenient and hygienci, rump wo...   
7  Lovely formula, but the bottle leaked in trans...   
8  The pump dispenses too much proudc each time. ...   
9  I saw miprove hydration bt not much improvemen...   

                                  spelling_anomalies  
0                                       [ve, nihtly]  
1                         [TThe, drspensses, prduct]  
2                 [ont, ocmbination, mdaee, shiiner]  
3  [pors, adn, causeed, sa, hydrationn, iuch, imp...  
4                     [Packoging, atd, ve, nighttty]  
5                            [clgoged, cused, aalso] 

# ✅ B. Méthodes statistiques : TF-IDF + N-grammes

In [ ]:
from collections import Counter
import re


# Charger la dataset
data = pd.read_csv("sephora_skincare_reviews.csv")

corpus = data['feedback'].dropna().tolist() 

test_sentence = "I just tried this new serum and it’s amazng!"

def detect_rare_words(corpus, test_sentence, seuil=1):
    words_in_corpus = re.findall(r'\b\w+\b', ' '.join(corpus).lower())
    corpus_freq = Counter(words_in_corpus)
    
    test_words = re.findall(r'\b\w+\b', test_sentence.lower())
    rare_words = [w for w in test_words if corpus_freq[w] <= seuil]
    
    print("📊 Fréquence dans le corpus :", dict(corpus_freq))
    return rare_words

print("\n🚨 Anomalies détectées :", detect_rare_words(corpus, test_sentence))


📊 Fréquence dans le corpus : {'noticed': 302, 'peeling': 98, 'when': 222, 'i': 1329, 'used': 542, 'it': 1347, 'with': 674, 'other': 110, 'active': 4, 'use': 499, 'sparingly': 61, 've': 273, 'been': 215, 'using': 221, 'nihtly': 5, 'tthe': 16, 'pump': 223, 'drspensses': 1, 'too': 127, 'much': 225, 'prduct': 6, 'each': 120, 'time': 108, 'not': 536, 'fro': 18, 'me': 479, 'sadly': 184, 'did': 115, 'ont': 12, 'suit': 90, 'my': 1892, 'oily': 98, 'ocmbination': 4, 'sin': 23, 'mdaee': 1, 'shiiner': 1, 'matches': 145, 'skin': 856, 'type': 191, 'well': 415, 'clogged': 82, 'pors': 3, 'adn': 27, 'causeed': 5, 'breakouts': 72, 'near': 106, 'jawline': 81, 'sa': 1, 'improved': 76, 'hydrationn': 3, 'but': 803, 'iuch': 1, 'imprvement': 1, 'in': 767, 'texture': 142, 'packoging': 1, 'is': 352, 'convenient': 71, 'atd': 2, 'hygienic': 87, 'works': 198, 'nighttty': 1, 'clgoged': 2, 'pores': 98, 'and': 1154, 'cused': 3, 'friend': 191, 'aalso': 5, 'the': 1138, 'change': 190, 'packaginng': 4, 'hygienci': 4, 'ru

# ✅ C. Méthodes avancées (NLP) – Transformers (BERT)

In [ ]:
from transformers import pipeline, AutoModelForMaskedLM, AutoTokenizer
import torch

def detect_with_bert_masking(sentence, target_word):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
    nlp = pipeline("fill-mask", model=model, tokenizer=tokenizer)

    masked_sentence = sentence.replace(target_word, tokenizer.mask_token)
    predictions = nlp(masked_sentence)
    top_predictions = [pred['token_str'] for pred in predictions]
    
    print("BERT propose :", top_predictions)
    if target_word not in top_predictions:
        return f"Anomalie détectée : '{target_word}' semble incohérent."
    return f"'{target_word}' est cohérent selon BERT."

sentence = "This serum makes my skin feel amazng."
detect_with_bert_masking(sentence, "amazng")



C:\Users\pc\anaconda3\envs\py38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not th

BERT propose : ['warm', 'better', 'cold', 'hot', 'sticky']


"Anomalie détectée : 'amazng' semble incohérent."

# ✅ (Bonus) Word Embeddings – Word2Vec (distance sémantique)

In [ ]:
from gensim.models import KeyedVectors
from gensim.downloader import load
from scipy.spatial.distance import cosine

model = load("word2vec-google-news-300")  
def detect_word2vec_anomaly(words):
    try:
        vectors = [model[word] for word in words if word in model]
        mean_vector = sum(vectors) / len(vectors)
        distances = {word: cosine(model[word], mean_vector) for word in words if word in model}
        anomaly = max(distances.items(), key=lambda x: x[1])
        return f"Anomalie potentielle : '{anomaly[0]}' (distance {anomaly[1]:.2f})"
    except:
        return "Certain(s) mot(s) absent(s) du vocabulaire Word2Vec."

print(detect_word2vec_anomaly(["serum", "skin", "amazng"]))


[======================================------------] 76.6% 1274.5/1662.8MB downloaded